For theoretical background please refer to theoretical discussion reading material on LMS .

The case study we take up in this module pertains to red wines. Dataset that we have contains chemical properties over 4000 red wines . In class we'll take up two important chemical properties which control wine's aroma and potency , namely sulphate and alcohol content.

We want to check how many groups exist within wines based on these chemicals. Lets begin.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

myfile='/Users/lalitsachan/Dropbox/March onwards/Python Data Science/Data/winequality-red.csv'

import pandas as pd
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

%matplotlib inline

wine=pd.read_csv(myfile,sep=";")

In [ ]:
wine=wine[["sulphates","alcohol"]]

In [ ]:
wine.head()

You can see that scale of sulphates and alcohol are very different. If we let them be like that , distances will be dominated by alcohol percentages alone due to their scale. We need to scale them

In [ ]:
wine_std=pd.DataFrame(scale(wine),columns=list(wine.columns))

Output of function `scale` is matrix , we need to convert it back to data frame in order to keep on using it as data frame.

In [ ]:
wine_std.head()

In [ ]:
wine_std.describe()

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6,7,8,9]

In the code below , we are trying different values of cluster numbers and for each we are examning silhoutte scores along with their silhoutte plots and visual representation of clusters.

Most of the code below has to do with how the plots are made pretty.

You dont need to do all that every time you do segmentation using K-means. This is more to do with giving you first hand experience as to what silhoutte plots are and how the eventual clusters look like.

You should also notice , how silhoutte scores both average and individual detriorate as we keep on unneccesarily increasing number of clusters. 

In [ ]:
X=wine_std.as_matrix()
for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhoutte score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors)

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1],
                marker='o', c="white", alpha=1, s=200)

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50)

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

    plt.show()

From silhoutte's score above we decided value of n_clusters as 3

In [ ]:
k = 3
kmeans = KMeans(n_clusters=k)
kmeans.fit(wine_std)

In [ ]:
labels = kmeans.labels_
wine_std["cluster"]=labels

In [ ]:
for i in range(k):
    # select only data observations with cluster label == i
    ds = wine_std[wine_std["cluster"]==i].as_matrix()
    # plot the data observations
    plt.plot(ds[:,0],ds[:,1],'o')

plt.show()

We can decide number of cluster using SSW as well as mentioned in the theoretical discussion. SSW is named `inertia_` as kmeans function's attribute.

In [ ]:
kmeans.inertia_

Lets look at inertia/SSW values for many Ks , see if our conclusions match with results obtained from silhoutte score.

In [ ]:
Ks=np.linspace(2,15,14)

In [ ]:
Ks

In [ ]:
ssw=[]
for k in Ks:
    kmeans=KMeans(n_clusters=int(k))
    kmeans.fit(wine_std)
    ssw.append(kmeans.inertia_)
plt.plot(Ks,ssw)

You can see here the clear elbow point at k=3.

We can try hierarchical clustering as well, it gives most efficient clusters but at much more computational cost. Lets see if results from it are very different from K-means.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
for n_clusters in range(2,10):
    cluster_model = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean',linkage='ward')
    cluster_labels = cluster_model.fit_predict(X)
    silhouette_avg = silhouette_score(X,cluster_labels,metric='euclidean')
    print("For n_clusters =", n_clusters, 
          "The average silhouette_score is:", silhouette_avg)


We have agreement on number of clusters. Lets see how the end results differ

In [ ]:
s = 3
hclust = AgglomerativeClustering(n_clusters=s, affinity='euclidean',linkage='ward')
hclust.fit(wine_std)

In [ ]:
labels = hclust.fit_predict(X)
wine_std["cluster"]=labels

In [ ]:
for i in range(s):
    # select only data observations with cluster label == i
    hc = wine_std[wine_std["cluster"]==i].as_matrix()
    # plot the data observations
    plt.plot(hc[:,0],hc[:,1],'o')

plt.show()

We can see that kmeans did an excellent job and we did not really need to get into hierarchical clustering to get the best results.

Next we look density based clustering algorithm dbscan. We will take a toy data to bring out clear difference between k-means and dbscan. This example demonstrate how k-means can no capture non-spherical groups in the data. We will be using make_moons function to make data which has two moon shaped very apparent groups. Lets see.

In [ ]:
from sklearn.datasets import make_moons

mydata = make_moons(n_samples = 2000,noise=0.05)
print(mydata[0].shape)
mydata=pd.DataFrame(mydata[0],columns=["X","Y"])
mydata.head()

In [ ]:
from ggplot import *

In [ ]:
ggplot(mydata,aes(x='X',y='Y'))+geom_point()

We can ask k-means to make as many clusters and it will oblige as opposed to dbscan which finds on its own how many natural clusters are there in the data

In [ ]:
kmeans=KMeans(n_clusters=2)
kmeans.fit(mydata)
mydata["cluster"]=kmeans.labels_
ggplot(mydata,aes(x='X',y='Y',color='cluster'))+geom_point()

As you can see , these are not the clusters that you expected . Lets ask k-mean to make 5 clusters and see what it does

In [ ]:
kmeans=KMeans(n_clusters=5)
kmeans.fit(mydata)
mydata["cluster"]=kmeans.labels_
ggplot(mydata,aes(x='X',y='Y',color='cluster'))+geom_point()

You can force k-means to make as many groups in the data as you want, it wont bother about how many natural groups there are in the data. lets see how dbscan treats this.

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics


In [ ]:
del mydata['cluster']

In [ ]:
db = DBSCAN(eps=0.2, min_samples=10, metric='euclidean').fit(mydata)
mydata['cluster']=db.labels_
ggplot(mydata,aes(x='X',y='Y',color='cluster'))+geom_point()

Lets try increasing eps and see what effects it has . 

In [ ]:
del mydata['cluster']
db = DBSCAN(eps=0.3, min_samples=10, metric='euclidean').fit(mydata)
mydata['cluster']=db.labels_
ggplot(mydata,aes(x='X',y='Y',color='cluster'))+geom_point()

As you can see increasing neighborhood size results in all points becoming part of same cluster, which is the intended effect.

Now lets see an example where , dbscan doesnt blindly club all the points into clusters, but leave points which are not supposed to be in any clusters. This also demonstrate how k-means is different from dbscan.

In [ ]:
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler


X = StandardScaler().fit_transform(X)

db = DBSCAN(eps=0.3, min_samples=10).fit(X)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

In [ ]:
df=pd.DataFrame(X,columns=['x1','x2'])
df['labels']=labels

In [ ]:
ggplot(df,aes(x='x1',y='x2',color='labels'))+geom_point()

You can see how dbscan has found outliers. You can play around with the values of eps and min_samples to see how that changes outlier detection and number of clusters detected by dbscan.
